In [1]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
dir <- "environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees"
suppressPackageStartupMessages(source("tree_helper_function.R"))
load("environment/brain_sim_nodtu/mode=gc_bias/mapDf.RData")
load(file.path(dir, "treeCons.RData"))
load(file.path(dir, "yAggConsNS.RData"))
load(file.path(dir, "treeCor.RData"))
load(file.path(dir, "yAggCorNS.RData"))
load(file.path(dir, "treeCons0.RData"))
load(file.path(dir, "yAggConsNS0.RData"))
load(file.path(dir, "treeMeanInf0.RData"))
load(file.path(dir, "yAggMeanInfNS0.RData"))
load(file.path(dir, "treeThr.RData"))
load(file.path(dir, "yAggThrNS.RData"))
suppressPackageStartupMessages(source("brain_simulation_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/tree_analysis/tree_filter.R"))

In [3]:
dfInf <- data.frame(matrix(nrow=0, ncol=4))
colnames(dfInf) <- c("Type", "Value", "nCuts", "gamma")

l <- length(treeCons$tip)
trees <- list(treeCons, treeCons0, treeMeanInf0, treeCor, treeThr)
names(trees) <- c("Consensus", "Consensus0", "MeanInf0", "Cor", "Thr")
yAgg <- list(yAggConsNS, yAggConsNS0, yAggMeanInfNS0, yAggCorNS, yAggThrNS)
names(yAgg) <- c("Consensus", "Consensus0", "MeanInf0", "Cor", "Thr")

dCutsDInf <- vector(mode = "list", length=length(yAgg))
gammas <- c(0.1,0.5,1,5,10)
names(dCutsDInf) <- paste("gamma=", gammas, sep="")
for(j in seq_along(dCutsDInf)) {
    gamma <- gammas[j]
    dCutsDInf[[j]] <- vector(mode="list", length(trees))
    names(dCutsDInf[[j]]) <- names(trees)
    for(i in seq(trees)) {
        globArr <- rep(-100, l+trees[[i]]$Nnode)
        descL <- sapply(Descendants(trees[[i]], seq(l+trees[[i]]$Nnode)), length)
        infReps <- assays(yAgg[[i]])[grep("infRep", assayNames(yAgg[[i]]))]
        infReps <- abind::abind(as.list(infReps), along = 3)
        obj <- mcols(yAgg[[i]])[["meanInfRV"]] + gamma*node.depth(trees[[i]],2)
        val <- findOptSum(trees[[i]], obj, l+1, descL)

        gc()
        dCutsDInf[[j]][[i]] <- findCuts(trees[[i]], globArr, obj, l+1, descL)
        gc()
        dfInf <- rbind(dfInf, data.frame(Type = names(trees)[i], Value = val,
               nCuts = length(dCutsDInf[[j]][[i]]), gamma=gamma))
    }
}
print(dfInf)
save(dfInf, file=file.path(dir, "dfInf.RData"))
save(dCutsDInf, file=file.path(dir, "dCutsDInf.RData"))

         Type     Value nCuts gamma
1   Consensus  15637.91 25470   0.1
2  Consensus0  15828.01 25800   0.1
3    MeanInf0  15890.42 25836   0.1
4         Cor  29087.02 26488   0.1
5         Thr  40503.04 36799   0.1
6   Consensus  57785.52 31097   0.5
7  Consensus0  57892.72 31195   0.5
8    MeanInf0  57988.46 31219   0.5
9         Cor  66824.81 31385   0.5
10        Thr  69159.26 37044   0.5
11  Consensus 100365.37 34445   1.0
12 Consensus0 100417.33 34440   1.0
13   MeanInf0 100525.01 34461   1.0
14        Cor 107280.27 34485   1.0
15        Thr 104401.29 37341   1.0
16  Consensus 348009.88 41520   5.0
17 Consensus0 348055.30 41521   5.0
18   MeanInf0 348058.35 41529   5.0
19        Cor 351919.53 41608   5.0
20        Thr 347650.82 41459   5.0
21  Consensus 606276.83 43412  10.0
22 Consensus0 606280.77 43414  10.0
23   MeanInf0 606290.92 43407  10.0
24        Cor 609011.68 43499  10.0
25        Thr 606207.02 43400  10.0


In [2]:
dCutsDInf

ERROR: Error in eval(expr, envir, enclos): object 'dCutsDInf' not found
